**Computation of Expectation Values**

This guide shows practical patterns to compute expectation values of the form

$⟨𝑂⟩_𝜓=⟨ψ∣O∣ψ⟩$

using ffsim, where
𝑂 can be a Hamiltonian or any other operator supported by `ffsim.linear_operator`.

We cover:

expectation values from a state vector,

the fast path when an operator provides a diagonal representation,

expectation values for operators given as a FermionOperator.

**Prerequisites**

We again use:

`norb`: number of spatial orbitals

`nelec = (n_alpha, n_beta)`: number of alpha, beta electrons

𝜓 is a state vector in the corresponding Fock basis sector

In ffsim, the dimension of the sector can be found via:

`ffsim.dim(norb, nelec)`.

In [48]:
import numpy as np
import ffsim

rng = np.random.default_rng(0)

norb = 4
nelec = (2, 2)
dim = ffsim.dim(norb, nelec)

# A random normalized complex state vector |psi>
psi = ffsim.random.random_state_vector(dim, seed=rng)
psi = psi / np.linalg.norm(psi)

print(dim, np.linalg.norm(psi))


36 1.0000000000000002


**Method 1: Use a LinearOperator**

Most operators in ffsim can be converted to a `scipy.sparse.linalg.LinearOperator` using:

`linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)`

Then compute

$⟨𝑂⟩_𝜓=⟨ψ∣(Oψ)⟩$

with `np.vdot`, which performs the conjugate dot product.

In [49]:
# Random but valid fermionic Hamiltonian
rng = np.random.default_rng(0)

H = ffsim.random.random_molecular_hamiltonian(
    norb=norb,
    seed=rng,
)

In [50]:
# Convert Hamiltonian to a LinearOperator
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)

# Expectation value
exp_val_linop = np.vdot(psi, H_linop @ psi).real

print("⟨ψ|H|ψ⟩ (LinearOperator):", exp_val_linop)

⟨ψ|H|ψ⟩ (LinearOperator): 5.616182913398914


**Notes**

`random_molecular_hamiltonian` generates a physically consistent one-body and two-body Hamiltonian.
For Hermitian operators, the expectation value should be real up to numerical error; you can take `expval.real` and it does not depend on how the operator is represented.
`LinearOperator` allows efficient matrix–vector products without constructing the full matrix.
The `FermionOperator` provides an explicit second-quantized representation of the Hamiltonian.

**Method 3: Start from a FermionOperator** 

If your operator is expressed as a `FermionOperator`, you can compute expectation values the same way:

Build a `FermionOperator` (e.g., from a Hamiltonian object), then

Convert it to a linear operator and compute $⟨ψ∣O∣ψ⟩$.

In [51]:
# Convert Hamiltonian to a FermionOperator
op = ffsim.fermion_operator(H)

# Convert FermionOperator to LinearOperator
op_linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)

# Expectation value of FermionOperator Method
expval_fermion = np.vdot(psi, op_linop @ psi).real

print("⟨ψ|H|ψ⟩ (FermionOperator method):", expval_fermion)

⟨ψ|H|ψ⟩ (FermionOperator method): 5.616182913398905


**Method 2 (fast path): Use the diagonal when available**


Some operators support an efficient diagonal extraction:

`diag = ffsim.diag(op, norb=norb, nelec=nelec)`

If you have the diagonal 𝑑 of 𝑂 in the chosen basis, then

$⟨𝑂⟩_𝜓=\sum_k 𝑑_𝑘 ∣𝜓_𝑘∣^2$.

This avoids forming `O @ psi` and is typically faster.
If the Hamiltonian is diagonalizable we can use this method.


In [53]:
rng = np.random.default_rng(0)

norb = 3
nelec = (2, 1)
dim = ffsim.dim(norb, nelec)

# Hartree–Fock state |psi>
psi = ffsim.hartree_fock_state(norb, nelec)

# Diagonal Coulomb Hamiltonian
one_body = ffsim.random.random_real_symmetric_matrix(norb, seed=rng)
diag_coulomb = ffsim.random.random_real_symmetric_matrix(norb, seed=rng)
diag_coulomb_mats = np.stack([diag_coulomb, diag_coulomb])
constant = 0.3

H = ffsim.DiagonalCoulombHamiltonian(
    one_body,
    diag_coulomb_mats,
    constant=constant,
)

# Method 1: LinearOperator
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)
expval_linop = np.vdot(psi, H_linop @ psi).real

# Method 2: Diagonal representation
# (only possible because H is diagonal in occupation basis)
diag = ffsim.diag(H, norb=norb, nelec=nelec)
expval_diag = np.vdot(psi, diag * psi).real

# Method 3: FermionOperator → LinearOperator
H_fermion = ffsim.fermion_operator(H)
H_fermion_linop = ffsim.linear_operator(H_fermion, norb=norb, nelec=nelec)
expval_fermion = np.vdot(psi, H_fermion_linop @ psi).real

# Results
print("Expectation values:")
print("LinearOperator:  ", expval_linop)
print("Diagonal:        ", expval_diag)
print("FermionOperator: ", expval_fermion)

Expectation values:
LinearOperator:   15.155450639407334
Diagonal:         15.155450639407338
FermionOperator:  15.155450639407338


**Expectation values for multiple states efficiently**

If you need expectation values for many state vectors $\{𝜓^{(𝑚)}\}$, you can reuse the same `LinearOperator` and loop.

For diagonal operators, you can reuse the diagonal vector and compute
$\sum_k 𝑑_𝑘 ∣𝜓{_𝑘 ^{(m)}}∣^2$ very easily.

In [46]:
norb = 3
nelec = (2, 1)
dim=ffsim.dim(norb, nelec)
rng = np.random.default_rng(0)

# Random Hamiltonian 
H = ffsim.random.random_molecular_hamiltonian(norb=norb, seed=rng)
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)

# Random state vectors
num_states = 5
psis = [ffsim.random.random_state_vector(dim, seed=rng) for _ in range(num_states)]
psis = [v / np.linalg.norm(v) for v in psis]

# --- Expectation values ---
expvals = [np.vdot(v, H_linop @ v) for v in psis] 

print(expvals)

[(-28.535247483817322+4.440892098500626e-15j), (-25.755236868551997+8.881784197001252e-16j), (-7.068801406274966+8.881784197001252e-16j), (2.5118636001750794+8.881784197001252e-16j), (-5.975245416403595+3.9968028886505635e-15j)]


In [47]:
import pandas as pd

# -----------------------------
# System definition
# -----------------------------
norb = 3
nelec = (2, 1)

# Hartree–Fock state |psi>
psi = ffsim.hartree_fock_state(norb, nelec)
psi /= np.linalg.norm(psi)

# -----------------------------
# Molecular Hamiltonian (SAFE way)
# -----------------------------
rng = np.random.default_rng(0)

H = ffsim.random.random_molecular_hamiltonian(
    norb=norb,
    seed=rng,
)

# -----------------------------
# LinearOperator representation
# -----------------------------
H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)
exp_linop = np.vdot(psi, H_linop @ psi).real

# -----------------------------
# FermionOperator representation
# -----------------------------
op = ffsim.fermion_operator(H)
H_linop_from_op = ffsim.linear_operator(op, norb=norb, nelec=nelec)
exp_fermion = np.vdot(psi, H_linop_from_op @ psi).real

# -----------------------------
# Table (for slides)
# -----------------------------
df = pd.DataFrame({
    "Representation": [
        "LinearOperator (MolecularHamiltonian)",
        "FermionOperator"
    ],
    "⟨ψ | H | ψ⟩": [
        exp_linop,
        exp_fermion
    ]
})

df.style.format({"⟨ψ | H | ψ⟩": "{:.6f}"})


,Representation,⟨ψ | H | ψ⟩
0,LinearOperator (MolecularHamiltonian),10.728417
1,FermionOperator,10.728417


**Summary**

Use `ffsim.linear_operator(op, norb, nelec)` + `np.vdot(psi, op @ psi)` for the `MolecularHamiltonian` since, it contains general two-body interactions and is not diagonal in the occupation-number basis.
If the operator provides a diagonal via `ffsim.diag`, computing $⟨𝑂⟩$ becomes a easy weighted sum of probabilities.
`FermionOperator` offers a clear second-quantized representation and integrates seamlessly into the same workflow.